<a href="https://colab.research.google.com/github/JanhaviJaiswar26/Online-Book-Store/blob/master/MovieRecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
movies = pd.read_csv('/content/Movies.csv')
credits = pd.read_csv('/content/MoviesCredits.csv')

In [ ]:
movies.head(10)

In [ ]:
credits.head(10)

In [ ]:
movies.shape

In [ ]:
credits.shape

In [ ]:
movies.info()

In [ ]:
credits.info()

In [ ]:
#merge both the adat set onto title

movies=movies.merge(credits,on='title')
movies

In [ ]:


#movies =movies.drop(['crew_y','movie_id_y','cast_y'],axis=1)
movies=movies.rename(columns={'crew_x':'crew','movie_id_x':'movie_id','cast_x':'cast'})


In [ ]:
#extracting the required columns

movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]
movies.head(10)

In [ ]:
#function for formatting the columns

import ast
def convert(text):
    a = []
    for i in ast.literal_eval(text):
        a.append(i['name'])
    return a

In [ ]:
movies.isna().sum()

In [ ]:
movies.dropna(inplace=True)


In [ ]:
movies['genres'] = movies['genres'].apply(convert)
movies.head()

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)
movies.head()

In [ ]:

ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [ ]:
#function to get the 3 most cast names

def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L

In [ ]:
movies['cast'] = movies['cast'].apply(convert)
movies.head()

In [ ]:
movies['cast'] = movies['cast'].apply(lambda x:x[0:3])
movies.head(10)

In [ ]:
#function for extraction only the director name form the list

def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)
movies.head(10)

In [ ]:
movies.sample(5)

In [ ]:
#function for removing the space from the names


def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [ ]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [ ]:
movies.head(10)

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies.head(5)

In [ ]:
#making tags from all the columns for specification


movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
movies.head(5)

In [ ]:
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])
new.head(5)

In [ ]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
vector = cv.fit_transform(new['tags']).toarray()

In [ ]:
vector.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)
similarity

In [ ]:
new[new['title'] == 'The Lego Movie'].index[0]

In [ ]:
def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)



In [ ]:
recommend('Avatar')

In [ ]:
import pickle
pickle.dump(new,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

# **Sentiment Analysis**

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df=pd.read_csv('/content/Reviews.csv')
df.head(1)

In [ ]:
df['review'][0]

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
#df=df.sample(10000)

In [ ]:
# forming in binary

df['sentiment'].replace({'positive':1,'negative':0},inplace=True)

In [ ]:
df.head(10)

In [ ]:
#testing code can be remove

import re
clean=re.compile('<.*?>')
re.sub(clean,'',df.iloc[2].review)

In [ ]:
#  Function for cleaning the html tags

def clean_html(text):
    clean=re.compile('<.*?>')
    return re.sub(clean,'',text)

In [ ]:
df['review']=df['review'].apply(clean_html)
df.head()

In [ ]:
#function to make all letters in lower case

def convert_lower(text):
    return text.lower()

In [ ]:
df['review']=df['review'].apply(convert_lower)
df.head(10)

In [ ]:
#function to remove special chareacters form the reviews

def remove_special(text):
    x=''
    for i in text:
        if i.isalnum():
            x=x+i
        else:
            x=x+' '
    return x

In [ ]:
df['review']=df['review'].apply(remove_special)
df.head(10)

In [ ]:
#removing the stop words

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')


In [ ]:
# shows all the stop words from that language

stopwords.words('english')

In [ ]:
#method 1
def remove_stopwords(text):
    x=[]
    for i in text.split():
        if i not in stopwords.words('english'):
            x.append(i)
    y=x[:]
    x.clear()
    return y

In [ ]:
#method 2 -good
from nltk.corpus import stopwords

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))  # Use set for faster lookup
    return [word for word in text.split() if word not in stop_words]

In [ ]:
df['review']=df['review'].apply(remove_stopwords)
df.head(10)

In [ ]:
#stemming means it take all similar words as coommon name from them

import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
#method 1 good
y=[]

def stem_words(text):
    for i in text:
        y.append(ps.stem(i))
    z=y[:]
    y.clear()
    return z

In [ ]:
#method2

from nltk.stem import PorterStemmer

def stem_words(text):
    ps = PorterStemmer()  # Instantiate the stemmer outside the loop
    return [ps.stem(word) for word in text]


In [ ]:
df['review']=df['review'].apply(stem_words)
df.head(10)

In [ ]:
#join funcuation

def join_back(list_input):
    return " ".join(list_input)

In [ ]:
df['review']=df['review'].apply(join_back)
df.head(10)

In [ ]:
df['review'][0]

In [ ]:
x=df.iloc[:,0:1].values
x.shape

In [ ]:
#method1

from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=1000)

In [ ]:
x=cv.fit_transform(df['review']).toarray()


In [ ]:
#method2


from sklearn.feature_extraction.text import CountVectorizer

# Configure CountVectorizer with options for speed and memory efficiency
cv = CountVectorizer(max_features=1000, stop_words='english')  # Adjust max_features as needed

# Fit and transform without converting to a dense array (keeps it as a sparse matrix)
x = cv.fit_transform(df['review'])


In [ ]:
x.shape

In [ ]:
x

In [ ]:
x[0].max()


In [ ]:
y=df.iloc[:,-1].values

In [ ]:
y.shape

In [ ]:
# splitting my data set into x and y i.e training set and testing set

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
x_test.shape

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB


In [ ]:
gcl=GaussianNB()
gcl.fit(x_train,y_train)

In [ ]:
y_pred_gcl=gcl.predict(x_test)
y_pred_gcl

In [ ]:
mcl=MultinomialNB()
mcl.fit(x_train,y_train)

In [ ]:
y_pred_mcl=mcl.predict(x_test)
y_pred_mcl

In [ ]:
bcl=BernoulliNB()
bcl.fit(x_train,y_train)

In [ ]:
y_pred_bcl=bcl.predict(x_test)
y_pred_bcl

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score

In [ ]:
print('GaussianNB',accuracy_score(y_test,y_pred_gcl))
print('GaussianNB',precision_score(y_test,y_pred_gcl))


In [ ]:
print('MultinomialNB',accuracy_score(y_test,y_pred_mcl))
print('MultinomialNB',precision_score(y_test,y_pred_mcl))

In [ ]:
print('BernoulliNB',accuracy_score(y_test,y_pred_bcl))
print('BernoulliNB',precision_score(y_test,y_pred_bcl))

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from textblob import TextBlob
import numpy as np

# Load review dataset
reviews = pd.read_csv('/content/Reviews.csv')

# Compute sentiment scores for each review
def get_sentiment(review):
    blob = TextBlob(review)
    return blob.sentiment.polarity

# Calculate sentiment scores and normalize
reviews['sentiment_score'] = reviews['review'].apply(get_sentiment)
reviews['normalized_sentiment'] = (reviews['sentiment_score'] + 1) / 2  # Normalize to [0,1]

# Create cosine similarity matrix based on `x` (the count vectorized review data)
content_similarity = cosine_similarity(x)

# Integrate sentiment into the recommendation function
def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = list(enumerate(content_similarity[index]))

    # Integrate with sentiment scores
    recommendations = []
    for i, score in distances:
        if i != index:  # Skip the same movie
            movie_title = new.iloc[i]['title']
            # Get the average sentiment for this recommended movie
            sentiment_score = reviews[reviews['movie_title'] == movie_title]['normalized_sentiment'].mean()
            # Adjust similarity score based on sentiment
            sentiment_weighted_score = score * sentiment_score
            recommendations.append((movie_title, sentiment_weighted_score))

    # Sort by combined score and return top recommendations
    recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)[:5]
    for rec in recommendations:
        print(rec[0])  # Display title of recommended movies
